In [2]:
import json
from sklearn.model_selection import train_test_split
f = open('train-v2.0.json')
data = json.load(f)

In [3]:
import pandas as pd
df=pd.read_csv('train_data.csv')
df.head()

,Unnamed: 0,Theme,Paragraph,Question,Answer_possible,Answer_text,Answer_start
0,2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,True,['2003'],[526]
1,6,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?,True,['Dangerously in Love'],[505]
2,7,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?,True,['Mathew Knowles'],[360]
3,8,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?,True,['late 1990s'],[276]
4,9,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?,True,['lead singer'],[290]


In [4]:
df_train ,df_dev=train_test_split(df, test_size=0.2, random_state=42,stratify=df['Answer_possible'])


In [5]:
data['data'][0]['paragraphs'][0]['qas'][0].keys()

dict_keys(['question', 'id', 'answers', 'is_impossible'])

In [6]:

j=0
res=[0,0,0]
for j in range(len(data['data'])):
    for k in range(len(data['data'][j]['paragraphs'])):
        cnt=0
        for i in range(len(data['data'][j]['paragraphs'][k]['qas'])):
            if data['data'][j]['paragraphs'][k]['qas'][i]['is_impossible']==True:
                print(data['data'][j]['paragraphs'][k]['qas'][i])
                res=[j,k,i]
                break
            else:
                cnt+=1

if(cnt==len(data['data'][j]['paragraphs'][k]['qas'])):
    print("None found")


{'plausible_answers': [{'text': 'action-adventure', 'answer_start': 128}], 'question': 'What category of game is Legend of Zelda: Australia Twilight?', 'id': '5a8d7bf7df8bba001a0f9ab1', 'answers': [], 'is_impossible': True}
{'plausible_answers': [{'text': 'Hyrule', 'answer_start': 67}], 'question': 'What land does Ocarina serve to protect?', 'id': '5a8d800edf8bba001a0f9abb', 'answers': [], 'is_impossible': True}
{'plausible_answers': [{'text': 'Game of the Year', 'answer_start': 278}], 'question': 'What accolade did Radar Princess receive after its release?', 'id': '5a8d8412df8bba001a0f9ac5', 'answers': [], 'is_impossible': True}
{'plausible_answers': [{'text': 'combat, exploration, and item collection', 'answer_start': 78}], 'question': 'What are the three main activities in The Legend of Zelda: Clawshot Princess?', 'id': '5a8d8513df8bba001a0f9acf', 'answers': [], 'is_impossible': True}
{'plausible_answers': [{'text': 'one', 'answer_start': 45}], 'question': 'Many different types of i

In [7]:
data['data'][0].keys()

dict_keys(['title', 'paragraphs'])

In [8]:
res

[441, 28, 0]

In [9]:
j,k,i=res
data['data'][j-1]['paragraphs'][k-10]['qas'][0].keys()

dict_keys(['plausible_answers', 'question', 'id', 'answers', 'is_impossible'])

In [10]:
data['data'][j-1]['paragraphs'][k-10]['qas'][0]['plausible_answers']

[{'text': 'PCI', 'answer_start': 173}]

In [11]:
df.iloc[0]['Paragraph']

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [12]:
def process_data(df):
    dict_final={}
    for i in range(df.shape[0]):
        dat=df.iloc[i]
        dict_final[dat["Theme"]]={}
    
    for i in range(df.shape[0]):
        dat=df.iloc[i]
        dict_final[dat["Theme"]][dat["Paragraph"]]=[]

    for i in range(df.shape[0]):
        dat=df.iloc[i]
        tmp={'Question':dat['Question'],'Answer_possible':dat['Answer_possible'],
                    'Answer_text':dat['Answer_text'],'Answer_start':dat['Answer_start'],'id':dat['Unnamed: 0']}
        tmp['Answer_text']=tmp['Answer_text'][1:-1]
        tmp["Answer_text"]=[tmp["Answer_text"]]
        # print("Type Answer is ",type(tmp["Answer_text"]))
        # print("Answer is ",tmp["Answer_text"][0])
        # break
        tmp['Answer_start']=tmp['Answer_start'][1:-1].split(',')
        if tmp['Answer_possible']==True:
            for i in range(len(tmp['Answer_start'])):
                tmp['Answer_start'][i]=int(tmp['Answer_start'][i])
        else :
            tmp['Answer_start']=[]
            tmp['Answer_text']=[]
        dict_final[dat["Theme"]][dat["Paragraph"]].append(tmp)


    cnt=[]
    def clean(dict_final):
        for i in dict_final.keys():
            for j in dict_final[i].keys():
                for k in dict_final[i][j]:
                    if not(len(k['Answer_text'])==len(k['Answer_start'])):
                        cnt.append(1)
                        print(len(k['Answer_text'])," ",len(k['Answer_start']))
                        print()
        print(len(cnt))

    clean(dict_final)


    final_json={'version':0.0,'data':[]}
    for i in dict_final.keys():
        tmp={'title':i,'paragraphs':[]}
        for j in dict_final[i].keys():
            tp={'qas':[],'context':(j[1:-1] if j[0]=='\'' else j)}
            for k in dict_final[i][j]:
                ty={}
                ty['question']=k['Question']
                ty['is_impossible']=(False if (k['Answer_possible']==True) else True)
                if ty['is_impossible']==True:
                    ty['plausible_answers']=[]
                ty['id']=str(k['id'])
                ty['answers']=[]
                if not(len(k['Answer_text'])==len(k['Answer_start'])):
                    print("Non Same at i = ",i," Para = ",j," question = ",k)
                    print(len(k['Answer_text']),"            ",len(k['Answer_start']))
                for l in range(len(k['Answer_text'])):
                    rr={}
                    rr['text']=(k['Answer_text'][l][1:-1] if k['Answer_text'][l][0]=='\'' else k['Answer_text'][l])
                    rr['answer_start']=int(k['Answer_start'][l])
                    ty['answers'].append(rr)
                tp['qas'].append(ty)
            tmp['paragraphs'].append(tp)
        final_json['data'].append(tmp)

    print("**********************Processed DF ****************** ")

    return final_json
    
            

            
            
            
            


        
    
    


In [13]:
import json
import numpy as np
def save_json(name,final_json):
    json_object = json.dumps(final_json, indent=4)
    
    # Writing to sample.json
    with open(name, "w") as outfile:
        outfile.write(json_object)
    return 
    class NpEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, np.integer):
                return int(obj)
            if isinstance(obj, np.floating):
                return float(obj)
            if isinstance(obj, np.ndarray):
                return obj.tolist()
            return super(NpEncoder, self).default(obj)

    json_object=json.dumps(final_json, cls=NpEncoder)
    with open(name, "w") as outfile:
        outfile.write(json_object)

In [14]:
save_json('test.json',process_data(df_train))
save_json('dev.json',process_data(df_dev))

0
**********************Processed DF ****************** 
0
**********************Processed DF ****************** 
